# Lab 3 (inference): Predict protein structures with ESMFold model on AWS HealthOmics

This notebook will guide you through using AWS HealthOmics to run protein structure inference against the ESMFold model and filter out low-quality predictions by comparing against a reference protein sequence

## Step 1: Setup and Configuration

First, let's get our AWS account information and set up variables we'll use throughout the notebook.

In [ ]:
%pip install py3Dmol
%pip install biotite 

In [ ]:
import boto3
import time
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import biotite.structure as struct
import biotite.structure.io.pdb as pdb
from utils.iam_helper import IamHelper
from utils.pdb_helper import structure_to_pdb_str
import py3Dmol
import pandas as pd

##########################################################

# Get AWS account information
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']
region = boto3.Session().region_name

# Define S3 bucket and folder names
S3_BUCKET = f'workshop-data-{account_id}'
LAB1_FOLDER = 'lab1-progen'
LAB2_FOLDER = 'lab2-amplify'
LAB3_FOLDER = 'lab3-esmfold'

print(f"Account ID: {account_id}")
print(f"Region: {region}")
print(f"S3 Bucket: {S3_BUCKET}")

##########################################################


## Step 2: Explore the AWS HealthOmics Ready2Run workflows 

We will first list all Ready2Run workflows to find out ESMFold workflow ID. We will then get the detailed information about the ESMFold workflow, including the required parameters.

### Step 2.1: List all Ready2Run workflows

In [ ]:
# Create HealthOmics client
omics = boto3.client('omics')

for workflow in omics.list_workflows(type='READY2RUN')['items']:
    print(f"ID: {workflow['id']},  Name: {workflow['name']}")

### Step 2.2: Get detailed information about the ESMFold workflow

In [ ]:
# ESMFold workflow ID
workflow_id = '1830181'

# Detailed information about ESMFold Ready2Run workflow
esmfold_workflow = omics.get_workflow(type='READY2RUN',id=workflow_id)
esmfold_workflow

### Step 2.3: Get the required parameters of the ESMFold workflow

In [ ]:
esmfold_workflow['parameterTemplate']

## Step 3: Predict protein structures using ESMFold

We will generate protein structures for all sequences selected at the previous step

### Step 3.1: Define variables and resources required for ESMFold runs

In [ ]:
# IAM role for ESMFold run
iam_helper = IamHelper()
job_role_arn = iam_helper.find_role_arn_by_pattern('OmicsWorkflowRole')

# File with the top sequence candidates from the previous step
top_sequence_candidates_file_path = f'./data/{LAB2_FOLDER}/top_sequence_candidates.fasta'

print(f'Job role ARN: {job_role_arn}') 

### Step 3.2: Load protein sequences and create ESMFold runs

In [ ]:
structures = []

# Load and process each protein sequence
for record in SeqIO.parse(top_sequence_candidates_file_path, "fasta"):

    fasta_file_name = f'{record.id}.fasta'

    # Store sequence in a separate fasta file
    with open(f'./data/{LAB3_FOLDER}/{fasta_file_name}', 'w') as f:
        SeqIO.write(record, f, "fasta")

    # Copy fasta file to S3
    !aws s3 cp ./data/$LAB3_FOLDER/$fasta_file_name s3://$S3_BUCKET/$LAB3_FOLDER/$fasta_file_name

    # wait for 10 seconds to prevent throttling 
    time.sleep(10)

    # Create ESMFold run
    response = omics.start_run(
        workflowType='READY2RUN',
        workflowId=workflow_id,
        name=f'esmfold-{record.id}',
        priority=100,
        roleArn=job_role_arn,
        outputUri=f's3://{S3_BUCKET}/{LAB3_FOLDER}/results',
        parameters={
        'fasta_path': f's3://{S3_BUCKET}/{LAB3_FOLDER}/{fasta_file_name}'
        }
    )

    run_id = response['id']
    structures.append({
        'run_id': run_id,
        'prompt_id': record.id,
        'sequence': str(record.seq),
        'description': record.description,
        'transform': None,
        'rmsd': None
    })

    print(f'Run ID: {run_id} sequence: {str(record.seq)} \n')



### Step 3.3: Check the status of the created ESMFold runs

In [ ]:
for s in structures:
    response = omics.get_run(id=s['run_id'])

    print(f"Run Name: {response['name']}")
    print(f"Run ID: {response['id']}")
    print(f"Status: {response['status']}")
    print(f"Parameters: {response['parameters']}")
    print()

## Step 4: Filter out protein sequences using the predicted structures 

### Step 4.1: Download the results of the ESMFold runs from the S3 output bucket

In [ ]:
!aws s3 cp s3://$S3_BUCKET/$LAB3_FOLDER/results ./data/$LAB3_FOLDER/results --recursive

### 4.2: View the reference structure and all predicted structures 

In [ ]:
view = py3Dmol.view(width=800, height=300, viewergrid=(1, len(structures) + 1))

# Add reference structure
with open(f'./data/reference.pdb','r') as f:
    view.addModel(f.read(), 'pdb', viewer=(0, 0)) 
    view.setStyle({'cartoon': {'color': 'blue'}}, viewer=(0, 0))
    view.zoomTo(viewer=(0, 0))


# Add all predicted structures
for i, s in enumerate(structures):
    run_id = s['run_id']
    pdb_file_name = f'./data/{LAB3_FOLDER}/results/{run_id}/out/pdb/prediction.pdb'

    # Add predicted structure
    with open(pdb_file_name,'r') as f:
        view.addModel(f.read(), 'pdb', viewer=(0, i+1)) 
        view.setStyle({'cartoon': {'color': 'green'}},viewer=(0, i+1))
        view.zoomTo(viewer=(0, i+1))
    
view.show()

### 4.3: Calculate RMSD for predicted structures in comparison to the reference one

In [ ]:
# Load reference structure
ref_structure = pdb.PDBFile.read(f'./data/reference.pdb').get_structure()[0]

# Process all predicted structures
for s in structures:

    # Get HealthOmics run ID
    run_id = s['run_id']

    # Load predicted structure
    pdb_file_name = f'./data/{LAB3_FOLDER}/results/{run_id}/out/pdb/prediction.pdb'
    gen_structure = pdb.PDBFile.read(pdb_file_name).get_structure()[0]

    # get CA atoms
    ref_ca = ref_structure[ref_structure.atom_name == 'CA']
    gen_ca = gen_structure[gen_structure.atom_name == 'CA']


    # Ensure the same length of the arrays of CA atoms
    min_len = min(len(ref_ca), len(gen_ca))
    ref_ca = ref_ca[:min_len]
    gen_ca = gen_ca[:min_len]    

    # calculate RMSD before superimpose
    rmsd_before = struct.rmsd(ref_ca, gen_ca) 

    # superimpose reference and generated structures by CA atoms
    gen_ca_fit,transform = struct.superimpose(ref_ca, gen_ca)
    s['transform'] = transform

    # calculate RMSD after superimpose
    rmsd_after = struct.rmsd(ref_ca, gen_ca_fit) 
    s['rmsd'] = rmsd_after

    print(f'Run ID: {run_id}')
    print(f'Min length: {min_len}')
    print("RMSD before superimpose: ", rmsd_before)
    print("RMSD after superimpose: ", rmsd_after)    
    print()

### Step 4.4: Sort protein sequences by RMSD 

In [ ]:
df = pd.DataFrame(structures)
df.set_index('prompt_id', inplace=True)
df = df.sort_values(by='rmsd',ascending=True)
df[['rmsd','run_id', 'sequence', 'description']]

### Step 4.5: Select three top sequences for the downstream analysis and save them in a FASTA file

In [ ]:
# Build a list of sequence records
records = []
for prompt_id, row in df.head(3).iterrows():
    record = SeqRecord(
        Seq(row.sequence),
        id=prompt_id,
        description=f'{row.description},rmsd={row.rmsd}'
    )
    records.append(record)

# Save the sequences in a  FASTA file
with open(f'./data/{LAB3_FOLDER}/top_sequence_candidates.fasta', 'w') as f:
    SeqIO.write(records, f, "fasta")

### 4.6: Superimpose and visualize the top predicted structure with the reference structure

In [ ]:
# Get top predicted structure info
top_gen_structure_info = df.iloc[0]

# Load top predicted structure
pdb_file_name = f'./data/{LAB3_FOLDER}/results/{top_gen_structure_info.run_id}/out/pdb/prediction.pdb'
gen_structure = pdb.PDBFile.read(pdb_file_name).get_structure()[0]

# Rotate predicted structure using transform
transform = top_gen_structure_info['transform']
gen_structure_fit = transform.apply(gen_structure)

# Visualize the reference and the top predicted structure
view = py3Dmol.view()

# add reference structure
ref_data = structure_to_pdb_str(ref_structure)
view.addModel(ref_data, 'pdb') 
view.setStyle({'model':0},{'cartoon': {'color': 'blue'}})

# add superimposed generated structure 
gen_data = structure_to_pdb_str(gen_structure_fit)
view.addModelsAsFrames(gen_data, 'pdb') 
view.setStyle({'model':1},{'cartoon': {'color': 'green'}})

view.zoomTo()
view.show()
